In [39]:
import pandas as pd
import psycopg2
from configparser import ConfigParser

### About
This 'sub-project' is about auditing and verifying all the data I'm going to use in future projects for my Pokemon MEGA Bank.

In [40]:
'''
Copied my config from my 'database-upload' file for ease-of-access.
'''

def config(filename="database.ini", section="postgresql"): # keeping filename for portability, changing later
    parser = ConfigParser() # creating parser
    parser.read(filename) # reading the .ini file
    db = {} # empty dictionary for database

    if parser.has_section(section): # checking if a config section exists
        params = parser.items(section) # "
        for param in params: # reading every setting
            db[param[0]] = param[1] # applying these for later use
        
    else:
        raise Exception("Section {0} not found in the {1} file".format(section, filename))
    
    try:
        conn = psycopg2.connect(**db) # connecting to the db by bypassing the dictionary
        print("Database connected successfully.")
    except:
        print("Database not connected successfully.")
        raise

    return conn

In [41]:
conn = config(filename=r"C:\Users\Jjoer\GitHub\Pokemon Stocks\database.ini") # connecting to db

df_cards = pd.read_sql_query("SELECT * FROM cards;", conn)
df_prices = pd.read_sql_query("SELECT * FROM prices;", conn)

conn.close() # closing the cursor


Database connected successfully.


C:\Users\Jjoer\AppData\Local\Temp\ipykernel_19068\3559632593.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cards = pd.read_sql_query("SELECT * FROM cards;", conn)
C:\Users\Jjoer\AppData\Local\Temp\ipykernel_19068\3559632593.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_prices = pd.read_sql_query("SELECT * FROM prices;", conn)


### 'Cards' table audit

The following steps are to verify that the 'cards' table is working as intended.

In [42]:
df_cards.head(10)

,card_id,name,supertype,subtypes,set_name,series,card_number,printed_total,artist,rarity
0,hgss4-1,Aggron,Pokémon,[Stage 2],HS—Triumphant,HeartGold & SoulSilver,1,102,Kagemaru Himeno,Rare Holo
1,xy5-1,Weedle,Pokémon,[Basic],Primal Clash,XY,1,160,Midori Harada,Common
2,pl1-1,Ampharos,Pokémon,[Stage 2],Platinum,Platinum,1,127,Atsuko Nishida,Rare Holo
3,dp3-1,Ampharos,Pokémon,[Stage 2],Secret Wonders,Diamond & Pearl,1,132,Kouki Saitou,Rare Holo
4,det1-1,Bulbasaur,Pokémon,[Basic],Detective Pikachu,Sun & Moon,1,18,MPC Film,Common
5,dv1-1,Dratini,Pokémon,[Basic],Dragon Vault,Black & White,1,20,Masakazu Fukuda,Rare Holo
6,mcd19-1,Caterpie,Pokémon,[Basic],McDonald's Collection 2019,Other,1,12,Sekio,None
7,pl3-1,Absol G,Pokémon,"[Basic, SP]",Supreme Victors,Platinum,1,147,Yusuke Ishikawa,Rare Holo
8,ex12-1,Aerodactyl,Pokémon,[Stage 1],Legend Maker,EX,1,92,Hajime Kusajima,Rare Holo
9,ex3-1,Absol,Pokémon,[Basic],Dragon,EX,1,97,Naoyo Kimura,Rare Holo


In [43]:
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19818 entries, 0 to 19817
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   card_id        19818 non-null  object
 1   name           19818 non-null  object
 2   supertype      19818 non-null  object
 3   subtypes       19818 non-null  object
 4   set_name       19818 non-null  object
 5   series         19818 non-null  object
 6   card_number    19818 non-null  object
 7   printed_total  19818 non-null  object
 8   artist         18679 non-null  object
 9   rarity         19515 non-null  object
dtypes: object(10)
memory usage: 1.5+ MB


### 'Prices' table audit

The following steps are to verify that the 'prices' table is working as intended.

In [44]:
df_prices.head(10)

,price_id,card_id,source,variant,condition_txt,updated_at,market_price,low_price,mid_price,high_price,raw_json,created_at
0,1116,dp5-19,tcgplayer,reverseHolofoil,Near Mint,2025-10-16,14.83,7.78,10.28,22.06,"{'low': 7.78, 'mid': 10.28, 'high': 22.06, 'ma...",2025-11-18 16:14:30.472139+00:00
1,1117,sm5-28,tcgplayer,normal,Near Mint,2025-10-16,0.19,0.09,0.21,2.00,"{'low': 0.09, 'mid': 0.21, 'high': 2.0, 'marke...",2025-11-18 16:14:30.472139+00:00
2,1118,sm5-28,tcgplayer,reverseHolofoil,Near Mint,2025-10-16,0.47,0.25,0.41,1.49,"{'low': 0.25, 'mid': 0.41, 'high': 1.49, 'mark...",2025-11-18 16:14:30.472139+00:00
3,1119,np-18,tcgplayer,holofoil,Near Mint,2025-10-16,14.27,8.01,11.26,18.49,"{'low': 8.01, 'mid': 11.26, 'high': 18.49, 'ma...",2025-11-18 16:14:30.472139+00:00
4,1120,sm75-26,tcgplayer,reverseHolofoil,Near Mint,2025-10-16,3.06,2.00,2.99,4.00,"{'low': 2.0, 'mid': 2.99, 'high': 4.0, 'market...",2025-11-18 16:14:30.472139+00:00
5,1121,sm75-26,tcgplayer,normal,Near Mint,2025-10-16,0.56,0.27,0.66,4.50,"{'low': 0.27, 'mid': 0.66, 'high': 4.5, 'marke...",2025-11-18 16:14:30.472139+00:00
6,1122,gym2-29,tcgplayer,1stEdition,Near Mint,2025-10-16,163.73,99.99,199.99,299.99,"{'low': 99.99, 'mid': 199.99, 'high': 299.99, ...",2025-11-18 16:14:30.472139+00:00
7,1123,gym2-29,tcgplayer,unlimited,Near Mint,2025-10-16,83.46,61.00,81.49,560.96,"{'low': 61.0, 'mid': 81.49, 'high': 560.96, 'm...",2025-11-18 16:14:30.472139+00:00
8,1124,xy5-26,tcgplayer,normal,Near Mint,2025-10-16,3.23,0.54,2.00,4.88,"{'low': 0.54, 'mid': 2.0, 'high': 4.88, 'marke...",2025-11-18 16:14:30.472139+00:00
9,1125,xy5-26,tcgplayer,reverseHolofoil,Near Mint,2025-10-16,11.42,2.96,15.61,49.99,"{'low': 2.96, 'mid': 15.61, 'high': 49.99, 'ma...",2025-11-18 16:14:30.472139+00:00


In [45]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196236 entries, 0 to 196235
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   price_id       196236 non-null  int64              
 1   card_id        196236 non-null  object             
 2   source         196236 non-null  object             
 3   variant        196236 non-null  object             
 4   condition_txt  196236 non-null  object             
 5   updated_at     196236 non-null  object             
 6   market_price   196023 non-null  float64            
 7   low_price      196235 non-null  float64            
 8   mid_price      196235 non-null  float64            
 9   high_price     196235 non-null  float64            
 10  raw_json       196236 non-null  object             
 11  created_at     196236 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(6)
memory usage: 18.0+ MB
